### Advanced Algorithms

Table of Content

1. Neural Networks
    1. Intuition
    2. Model
    3. Tensorflow
    4. Python Implementation

2. Vectorization

3. Neural Network Training
    1. Activation Function
    2. Multiclass Classification
    3. Advanced Optimization & Layering

4. Back Propagation

5. Bias & Variance

6. ML Development Process

7. Skewed Dataset

8. Decision Trees
    1. Decision Tree Learning
    2. Tree Ensembles

#### Decision Trees

Nomenclature
1. root node -> first node
2. decision nodes -> nodes after root 
3. purity -> % of output classification is classified correctly 


Learning Process of Decision Trees
1. decision 1 -> decisions to make for trees
    1. how to choose which feature to use at which node?
        1. max purity
2. decision 2 -> when do you stop splitting? (model hyper params)
    1. when node is 100% one class
    2. when splitting a node will result in the tree exceeds max depth
    3. when improvments in purity score are below threshold 
    4. when number of examples in a node is below a threshold


Entropy
1. impurity
2. a function that measures impurity
3. you use it to eval how well a node classifies data
4. intuition
    1. whithin a set there are 2 categories, pure/impure
    2. a ratio is created to determine weather the set is impure (50/50 split) or pure (100/0, 0/100 split)
    3. so this ratio is in essenece the p that goes in to the entropy function

5. entropy is denoted as H(p) where p= fraction of examples that are true
6. function is
    1. H(p) = -p* log(p) - (1-p) * log(1-p)
        0 is pure, 1 is impure. 

Getting from Entropy to Information Gain
1. quntification on how effective a node is splitting the data so that it becomes more/less pure
2. intution
    1. you have the entropy value of the data before a node splits, and the entropy value for the subset of data after the split occurs.
    2. you take the weighted entropy value of the post split values and compare it to the pre-split value

3. Mathmatically 
    1. information gain = H(p_root) - w_left * H(p_left) + w_right * H(p_right)
    2. so in essence 
        1. information gain = root branch entropy - weighted avg sub branch entropy
        2. since entropy 1 is impure, and 0 means pure, improvment in entropy (information gain)
            1. information gain > 0

Creating bools rom different variables
1. one-hot encoding
    1. bascially create multiple bool from cat variable, by doing 
        1. cat 1 = true, not cat 1 = false. 2nd bool = cat 2 = true, not cat 2 = false repeat to n categories
        2. in python `df = pd.getdummies(data = df, prefix = cat_variables, coumns = cat_variables)

2. continuous valued features
    1. bascially create a numeric threshold
    2. true if greater/lesser than threshold, vice versa

Regression Trees
1. x variables is are bools, but y is float
2. split using nodes, then leaf node is the average y value within the leaf dataset
3. goal of regression tree -> each split results in a smaller variance within the subset

Info Gain in Regression Tress
1. intuition -> same as bin classification except instead of using entropy use variance. 
    1. Information gain = root_var - (w_left * left_subset_var + w_right * right_subset_var)
        1. w_left & w_right -> count(subset)/count(root)


Using Multiple Decision Trees
1. weakness of single decision trees
    1. highly sensitive
    2. tree ensamble 
        1. multiple decision trees to address sensitivity

2. the Ensambles way
    1. multiple trees with different methods of creating the tree
    2. each model will predict based on input x
    3. then you get trees to vote
    4. take the average vote


Building Tree Ensemble
1. Bagged Decision Trees 
    1. Bootstrapping Data
        1. create multiple training set from original dataframe
        2. sample with replacment and sample n observations where n = len(original_data)
        3. n sample dataset = n trees in ensemble
    2. determining number of trees
        1. typically between 64-128, usually 100
        2. dim return after 130

2. Random Forests
    1. Bagged Decision Trees + random feature selection
    2. Random Freature selction
        1. 1 more layer of complexity, where you don't use all feature variables, but n sample features (without replacment) 
        2. random feature selection occurs on every level of node. meaning choose random k features at root, then another random feature selection in next node etc.
    3. done on every tree
    4. each tree is independent 

3. XGBoost
    1. random forest + reinforcement learning
    2. Reinforcement Learning
        1. after creating the first tree, you modify the probability of which observations to choose in the bootstrap of sampled dataset for tree creation
        2. make observations that are missclassified more likely to be sampled in random sampling in boostrapped datset
    3. Trees are dependent of previous tree
    
4. Modeling Process of XGBoost
    1. split train test set
    2. split train set to fit and eval
    3. so the overall data split structure
        1. main
            1. test (10-30%)
            2. train (70-90%)
                1. fit (10-20% of train)
                2. eval (80-90% of train)

    4. what the split does
        1. test -> test the entire, once the model is constructed
        2. train -> data used to train the data
            1. fit -> data actually used to fit (train) the model. use this data to min cost. 
            2. eval -> smaller subset during training to eval model performance for each iteration. 
                1. this exists because it is an ensamble of multiple trees
                2. after each iteration of tree creation the aggregate tree ensemble's performance is tested against the evaluation set
                3. if the performance on eval set improves, training continues. if performance stagnates/worsens, model stops. 

    5. Eval set in practice
        1. Let’s say you are tuning the model and testing different hyperparameters like learning rate, tree depth, and regularization:
            1. You train the model with a learning rate of 0.3, max depth of 6, and regularization strength of 1.0.
            2. After each boosting iteration, you evaluate the performance on the eval set.
            3. If you see that the performance on the training set is improving while the eval set starts deteriorating, this indicates overfitting. You might then:
            4. Decrease the learning rate to 0.1 to slow down the learning process.
            5. Reduce the max tree depth to 4 to prevent overfitting.
            6. Increase regularization (e.g., lambda from 1.0 to 2.0) to reduce model complexity.

##### In Python

Starting from the Training dataset
1. `N = int(len(X_train)*0.8)` # i.e. 80% the count of 80% of observations is saved as value N
2. `X_train_fit, X_train_eval, y_train_fit, y_train_eval = X_train[:N], X_train[N:], y_train[N:], y_train[:N]` # train data is split between fit set and eval set, with observations below index value N being part of fit set and above N being part of eval set
 
Modeling
`xgb_model = XGBClassifier(n_estimators = 500, learning_rate = 0.1, verbosity = 1, random_state = RANDOM_STATE)`
`xgb_model.fit(X_train_fit, y_train_fit, eval_set[(X_train_eval, y_train_eval)], early_stopping_rounds = 10)`
1.  when fitting the model with the data, you need to feed it 4 datasets, X_fit & eval, y_fit & eval.
2.  the early_stopping_rounds parameter helps to stop the model training if its eval metric is no longer improving on the validation set.
 
Model Results
1. Gives you model with the best hyper parameters.
2. `gb_model.best_iteration` # gives you the optimized number of trees in the forest.
3. Then you do a final test of model against the test set
 
 
Classification Trees
`from xgboost import XGBClassifier`
`model = XGBClassifier()`
`model.fit(X_train, y_train)`
`y_pred = model.predict(X_test)`

Regressed Trees
`from xgboost import XGBRegressor`
`model = XGBRegressor()`
`model.fit(X_train, y_train)`
`y_pred = model.predict(X_test)`
 
When to use Decision Trees
1. When using Structured(Tabular) Data not on unstructured data
2. When you need to train a model quickly
3. Small decision trees are human interpretable
